In [1]:
#default_exp interference

In [2]:
import sys
#sys.path.append("/home/qhs67/git/bachelorthesis_sven_thaele/code/")
sys.path.insert(0, "../pointpillars")

In [3]:
#export
import torch
import logging

from data.dataset import VelTrainDataset, collate_wrapper
from modules.pointpillars import PointPillars, init_weights
from utils.viewer.pcviewer import PCViewer
from utils.io import read_config, \
                     load_network_save, \
                     load_single_pcd, \
                     write_list_to_file
from utils.math.box_ops import bb_tensor_to_velodyne_coords, \
                               convert_boxes_to_3d_corners, \
                               vel_point_to_left_cam_point

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
#export
logger = logging.getLogger(__name__)


In [5]:
#export
def _evaluate_network(preds: tuple, file_name: str, out_folder: str, calib_file: str ):
    """
    Helper method for evaluate_network. Calculates all required attributes for each prediction
    and writes them to the disk.

    :param preds: Gives the predictions for a single point (no batch dimension).
                    (pred_occ, pred_cls, pred_head, pred_box)
    :param file_name: Gives the file name for the given pcloud
    :param out_folder: String giving the path to the folder into which all output text files should be stored
    :param calib_file:

    """

    # S1: get data from network (xyz, hwl, theta)
    # S2: calc 2D box with box_ops method
    # S3: write data to corresponding file name

    pred_occ, pred_cls, pred_head, pred_boxes = preds
    pred_box_corners = convert_boxes_to_3d_corners(pred_boxes, batch_dim=False)

    lines = []

    #x_min, y_min, x_max, y_max for 2d boxes in camera coords
    for i, pred_box in enumerate(pred_boxes):
        one = torch.tensor([1], device="cuda:0")
        xyzv_min = torch.cat((pred_box_corners[i, :3], one), dim=0)
        xyc_min = vel_point_to_left_cam_point(xyzv_min, calib_file)
        xyzv_max = torch.cat((pred_box_corners[i, 3:], one), dim=0)
        xyc_max = vel_point_to_left_cam_point(xyzv_max, calib_file)
        
        cls = "Car"
        # y orientation is reversed (pixel 0 starts at top,
        # see https://github.com/windowsub0406/KITTI_Tutorial/blob/master/velo2cam_projection_detail.ipynb)
        x_min, y_min, x_max, y_max = xyc_min[0], xyc_max[1], xyc_max[0], xyc_min[1]
        xv, yv, zv, h, w, l, theta = pred_box[0], pred_box[1], pred_box[2], pred_box[3], pred_box[4], pred_box[5], pred_box[6]

        """
        #Values    Name      Description
        ----------------------------------------------------------------------------
           1    type         Describes the type of object: 'Car', 'Van', 'Truck',
                             'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram',
                             'Misc' or 'DontCare'
           1    truncated    Float from 0 (non-truncated) to 1 (truncated), where
                             truncated refers to the object leaving image boundaries
           1    occluded     Integer (0,1,2,3) indicating occlusion state:
                             0 = fully visible, 1 = partly occluded
                             2 = largely occluded, 3 = unknown
           1    alpha        Observation angle of object, ranging [-pi..pi]
           4    bbox         2D bounding box of object in the image (0-based index):
                             contains left, top, right, bottom pixel coordinates
           3    dimensions   3D object dimensions: height, width, length (in meters)
           3    location     3D object location x,y,z in camera coordinates (in meters)
           1    rotation_y   Rotation ry around Y-axis in camera coordinates [-pi..pi]
           1    score        Only for results: Float, indicating confidence in
                             detection, needed for p/r curves, higher is better."""
        # TODO: add other prediction types

        line = f"{cls} -1 -1 0 {x_min} {y_min} {x_max} {y_max} {h} {w} {l} {-zv} {-yv} {xv} {theta} {pred_occ[i]}"
        lines.append(line)

    write_list_to_file(lines, out_folder + file_name)



In [6]:
#export
def evaluate_network(eval_folder: str,
                     label_folder: str,
                     net_save_name: str,
                     out_folder: str,
                     calib_folder: str):
    """
    Evaluates a saved network. This means it creates text files for each prediction pcloud in the given
    out_folder location. These text files contain the necessary information which are required by the KITTI
    evaluation script found in the object development kit (http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=3d).

    :param eval_folder: String giving the path to the eval folder, which contains the pclouds which should be evaluated
    :param label_folder: String giving the path to the label folder, which contains the corresponding labels.
                            The labels in eval_folder and label_folder have to match and fit the KITTI labeling convention.
    :param net_save_name: String giving the network save name in the checkpoint folder.
    :param out_folder: String giving the path to the output folder, into which all the txt files will be exported.
    :param calib_folder:
    """
    batch_size = 1
    dataset = VelTrainDataset(eval_folder, label_folder)
    dataloader = torch.utils.data.DataLoader(dataset,
                                             batch_size=batch_size,
                                             num_workers=0,
                                             collate_fn=collate_wrapper,
                                             pin_memory=False,
                                             shuffle=False)
    conf = read_config()
    with torch.no_grad():
        net_save = load_network_save(net_save_name)
        net = PointPillars(conf, train_mode=False)
        net.load_state_dict(net_save)
        net.eval()
        net.cuda()

        for i, batch in enumerate(dataloader):
            file_name = dataset.labels[i]

            pil_batch, ind_batch, label_batch, mask_batch = batch
            preds = net(pil_batch, ind_batch, label_batch)
            # TODO: add calib file location
            _evaluate_network(preds, file_name, out_folder, f"{calib_folder}/{file_name}")
            print(f"Item: {i} done.")



In [7]:
#export
def predict_single_pcd(pcd_nbr: int, net_save_name: str):
    """Create a network prediction from a saved network for a single point cloud"""

    vel_folder = "/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/velodyne/training"
    label_folder ="/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/label_2/training"

    dataset = VelTrainDataset(vel_folder, label_folder)
    batch_tuple = dataset[pcd_nbr]
    
    batch = []
    for i, tensor in enumerate(batch_tuple):
        tensor = tensor.unsqueeze(0)
        batch.append(tensor)
        
    pil_batch, ind_batch, label_batch = batch

    with torch.no_grad():
        conf = read_config()
        net_save = load_network_save(net_save_name)
        net = PointPillars(conf, train_mode=False)
        net.load_state_dict(net_save)
        net.eval()
        net.cuda()

        _ ,_ ,_ , pred_box = net(pil_batch, ind_batch, label_batch)

    return pred_box, label_batch

In [8]:
#export
def view_single_pcd_prediction(pcd_nbr: str, net_save_name: str):
    pcloud = load_single_pcd(pcd_nbr)
    pred_anchors, label_batch= predict_single_pcd(int(pcd_nbr), net_save_name)

    label_batch = label_batch[0]
    label_batch = bb_tensor_to_velodyne_coords(label_batch)

    PCViewer.view_pcd_from_network_output(pcloud, pred_anchors, label_batch)



In [9]:
pcloud = load_single_pcd('002137')
pcloud

tensor([[ 7.7512e+01,  4.7000e-02,  2.8300e+00,  0.0000e+00],
        [ 5.2519e+01,  8.8510e+00,  2.0100e+00,  1.6000e-01],
        [ 5.2292e+01,  8.9810e+00,  2.0030e+00,  1.0000e-01],
        ...,
        [ 3.8420e+00, -1.4470e+00, -1.7980e+00,  3.9000e-01],
        [ 3.8530e+00, -1.4380e+00, -1.8020e+00,  0.0000e+00],
        [ 3.8750e+00, -1.4320e+00, -1.8100e+00,  0.0000e+00]], device='cuda:0')

In [10]:
torch.min(pcloud, dim=0), torch.max(pcloud, dim=0)

(torch.return_types.min(
 values=tensor([-71.8370, -55.1200,  -8.8140,   0.0000], device='cuda:0'),
 indices=tensor([ 13281,   1474, 113923,      0], device='cuda:0')),
 torch.return_types.max(
 values=tensor([79.8020, 32.9710,  2.8300,  0.9900], device='cuda:0'),
 indices=tensor([ 6116, 10917,     0, 28486], device='cuda:0')))

In [11]:
#export
#view_single_pcd_prediction("002152", "trained_22-06-2021-19-43-55_e10000")
#view_single_pcd_prediction("000015", "trained_28-06-2021-13-18-04_e80")

In [ ]:
vel_folder = "/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/velodyne/training"
label_folder ="/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/label_2/training"
out_folder = "/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/evaluation/training/"
calib_folder = "/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/calib"
net_save_name = "trained_28-06-2021-13-18-04_e80"

evaluate_network(vel_folder, label_folder, net_save_name, out_folder, calib_folder)

INFO - 2021-07-22 14:53:45,706 - head - Initialize detection head...
INFO - 2021-07-22 14:53:45,709 - boxmatch - Initializing BoxMatch module...
INFO - 2021-07-22 14:53:46,407 - head - Forward through detection head...
INFO - 2021-07-22 14:53:46,409 - boxmatch - Calculating absolute_boxes_from_anchors...
INFO - 2021-07-22 14:53:46,955 - head - Forward through detection head...
INFO - 2021-07-22 14:53:46,956 - boxmatch - Calculating absolute_boxes_from_anchors...


Item: 0 done.


INFO - 2021-07-22 14:53:48,068 - head - Forward through detection head...
INFO - 2021-07-22 14:53:48,070 - boxmatch - Calculating absolute_boxes_from_anchors...


Item: 1 done.
Item: 2 done.


INFO - 2021-07-22 14:53:48,211 - head - Forward through detection head...
INFO - 2021-07-22 14:53:48,213 - boxmatch - Calculating absolute_boxes_from_anchors...
INFO - 2021-07-22 14:53:48,365 - head - Forward through detection head...
INFO - 2021-07-22 14:53:48,367 - boxmatch - Calculating absolute_boxes_from_anchors...


...

INFO - 2021-07-22 15:05:33,842 - head - Forward through detection head...
INFO - 2021-07-22 15:05:33,844 - boxmatch - Calculating absolute_boxes_from_anchors...


Item: 890 done.


INFO - 2021-07-22 15:05:34,975 - head - Forward through detection head...
INFO - 2021-07-22 15:05:34,978 - boxmatch - Calculating absolute_boxes_from_anchors...


Item: 891 done.
Item: 892 done.


INFO - 2021-07-22 15:05:35,115 - head - Forward through detection head...
INFO - 2021-07-22 15:05:35,116 - boxmatch - Calculating absolute_boxes_from_anchors...
